In [1]:
import json
import os
from pprint import pprint

### 수정해야할 부분

In [2]:
model_name__ = 'badminton_300(shoes).h5'   # 수정
root_dir = '/home/joon/dataset'  # 수정
gt_save_path = os.path.join(root_dir, 'ground_truth')
if not os.path.isdir(gt_save_path):
    os.mkdir(gt_save_path)
dr_save_path = os.path.join(root_dir, 'detection_results', '{}'.format(model_name__[:-3]))   # 오류시 디렉토리 생성
if not os.path.isdir(dr_save_path):
    os.mkdir(dr_save_path)

In [3]:
data_root_dir = '/home/joon/dataset'   # 수정
data_dir = 'test'

### json files & class ids

In [4]:
data_path = os.path.join(data_root_dir, data_dir)
annotations_path = os.path.join(data_root_dir, 'annotations')
test_anno_path = os.path.join(annotations_path, 'instances_test.json')
train_anno_path = os.path.join(annotations_path, 'instances_train.json')
val_anno_path = os.path.join(annotations_path, 'instances_val.json')

In [5]:
loaded = json.load(open(test_anno_path))
class_ids_info = {i['id']:i['name'] for i in loaded['categories'] }
class_ids_info

{1: 'player', 2: 'shoes'}

### Ground-truth (Mask RCNN -> YOLO)

In [6]:
matched = {}
for label_info in loaded['annotations']:
    cate_id = label_info['category_id']
    class_ = class_ids_info[cate_id]
    img_id = label_info['image_id']
    img_name = loaded['images'][img_id-1]['file_name']
    bbox = label_info['bbox']
    class_n_bbox = [class_, str(int(bbox[0])), str(int(bbox[1])), str(int(bbox[0])+int(bbox[2])), str(int(bbox[1])+int(bbox[3]))]
    if img_name not in matched:
        matched[img_name] = [class_n_bbox]
    else:
        matched[img_name].append(class_n_bbox)

for img, gt_bboxes_info in matched.items():
    gt_file_name = img.replace('jpg', 'txt')
    gt_file_path = os.path.join(gt_save_path, gt_file_name)
    writing = '\n'.join([' '.join(gbi) for gbi in gt_bboxes_info ])
    with open(gt_file_path, 'w') as f:
        f.write(writing)

### Detection results (Mask RCNN -> YOLO)

In [9]:
import badminton   # 가상환경 이름
config = badminton.CocoConfig() # 기본 설정들. hyperparameter

DATA_DIR = "/home/joon/dataset/test/"  # TODO: enter value here
model_name = model_name__ 

In [10]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]='0' # 그래픽카드 번호

import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import PIL 

from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

%matplotlib inline 

# root: 현재 폴더
ROOT_DIR = os.path.abspath("./")
sys.path.append(ROOT_DIR) 

# model 들어있는 폴더
MODEL_DIR = os.path.join(ROOT_DIR, "models")

# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
#config.display()

DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0
TEST_MODE = "inference"

def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

dataset = badminton.CocoDataset()
dataset.load_coco(data_root_dir, "test") # test로 바꾸기

# Must call before using the dataset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

# Load weights
model_path = os.path.join(MODEL_DIR, model_name)

print("Loading weights ", model_path)
model.load_weights(model_path, by_name=True)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Images: 60
Classes: ['BG', 'player', 'shoes']
Loading weights  /mnt/c/Users/korea/Documents/badminton/mask-rcnn/models/badminton_300(shoes).h5


In [12]:
import cv2

started = time.time()
for i, image_name in enumerate([f for f in sorted(os.listdir(data_path)) if 'jpg' in f]):
    
    image_path = os.path.join(data_path, image_name)
    original_image = cv2.imread(image_path, cv2.IMREAD_COLOR)

    # Run object detection
    results = model.detect([original_image], verbose=0)
    r = results[0]
    
    writing = []
    for roi, c_id, score in zip(r['rois'], r['class_ids'], r['scores']):
        writing.append(' '.join([class_ids_info[c_id], 
                                 str(score), str(int(roi[1])), str(int(roi[0])), str(int(roi[3])), str(int(roi[2]))]))
    
    write_input = '\r\n'.join(writing)
    save_path = os.path.join(dr_save_path, image_name.replace('jpg', 'txt'))
    
    with open(save_path, 'w') as f:
        f.write(write_input)
    
    if i % 10 == 0:
        print ('{}, elapsed : {:.3}m'.format(i, (time.time()-started)/60))

1
0, elapsed : 0.0605m
1
1
1
1
1
1
1
1
1
1
10, elapsed : 0.661m
1
1
1
1
1
1
1
1
1
1
20, elapsed : 1.28m
1
1
1
1
1
1
1
1
1
1
30, elapsed : 1.94m
1
1
1
1
1
1
1
1
1
1
40, elapsed : 2.59m
1
1
1
1
1
1
1
1
1
1
50, elapsed : 3.19m
1
1
1
1
1
1
1
1
1


In [14]:
loaded['images']

[{'id': 1,
  'file_name': '000001.jpg',
  'width': 1920,
  'height': 1080,
  'date_captured': '2020-03-31 18:12:54.454204',
  'license': 1,
  'coco_url': '',
  'flickr_url': ''},
 {'id': 2,
  'file_name': '000002.jpg',
  'width': 1920,
  'height': 1080,
  'date_captured': '2020-03-31 18:12:54.454204',
  'license': 1,
  'coco_url': '',
  'flickr_url': ''},
 {'id': 3,
  'file_name': '000003.jpg',
  'width': 1920,
  'height': 1080,
  'date_captured': '2020-03-31 18:12:54.454204',
  'license': 1,
  'coco_url': '',
  'flickr_url': ''},
 {'id': 4,
  'file_name': '000004.jpg',
  'width': 1920,
  'height': 1080,
  'date_captured': '2020-03-31 18:12:54.454204',
  'license': 1,
  'coco_url': '',
  'flickr_url': ''},
 {'id': 5,
  'file_name': '000005.jpg',
  'width': 1920,
  'height': 1080,
  'date_captured': '2020-03-31 18:12:54.454204',
  'license': 1,
  'coco_url': '',
  'flickr_url': ''},
 {'id': 6,
  'file_name': '000006.jpg',
  'width': 1920,
  'height': 1080,
  'date_captured': '2020-03-31

In [15]:
pprint (loaded['annotations'][0])

{'area': 662,
 'bbox': [1208.0, 567.0, 32.0, 37.0],
 'category_id': 2,
 'height': 1080,
 'id': 1,
 'image_id': 1,
 'iscrowd': 0,
 'segmentation': [[1218.0,
                   603.5,
                   1212.0,
                   603.5,
                   1207.5,
                   595.0,
                   1211.5,
                   589.0,
                   1210.5,
                   576.0,
                   1214.0,
                   575.5,
                   1220.0,
                   566.5,
                   1222.5,
                   573.0,
                   1235.0,
                   579.5,
                   1239.5,
                   587.0,
                   1218.0,
                   603.5]],
 'width': 1920}
